# langGraph Practice 1



![구성도](/home/ansgyqja/AI_application/images/9.structure_practice_1.png)

In [1]:
# API 키를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API 키 정보 로드
load_dotenv()

True

In [2]:
# LangSmith 추적을 설정합니다. https://smith.langchain.com
# !pip install -qU langchain-teddynote
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("langGraph_practice_1")

LangSmith 추적을 시작합니다.
[프로젝트명]
langGraph_practice_1


In [3]:
from typing import Annotated,TypedDict,List,Literal
from langchain_core.output_parsers import StrOutputParser
from langgraph.graph.message import add_messages
from langgraph.graph import StateGraph, START, END
from langchain_teddynote.evaluator import GroundednessChecker
from langchain import hub
from operator import itemgetter
from langchain_teddynote.messages import messages_to_history
from langchain_teddynote.tools.tavily import TavilySearch
from langgraph.checkpoint.memory import MemorySaver

import sys, os
sys.path.append(os.path.abspath(os.path.join(os.path.dirname('get_naver_news'), '..')))
from module.utils import re_write_prompt,get_gemini,format_docs
from module.get_local_pdf import rag_pdf

In [4]:
class State(TypedDict):
    question: Annotated[list, add_messages]  # 질문(누적되는 list)
    context : Annotated[str,'Context']
    answer : Annotated[str,'Answer']
    relevant : Annotated[str,'yes or no']
    chat_history:Annotated[list,add_messages]

# re write node
def re_write_node(state : State) -> State:
    chain = re_write_prompt | get_gemini() | StrOutputParser()
    response = chain.invoke({"question": state['question']})
    return State({"question":response})

# retriever node
def retriever_node(state : State) -> State:
    retriever =rag_pdf().get_retriever()
    latest_question = state["question"][-1].content
    document = retriever.invoke(latest_question)
    context = format_docs(document)
    return State({"context":context})

def retriever_relevant(state : State):
    question_answer_relevant = GroundednessChecker(
        llm=get_gemini(), target="question-retrieval"
    ).create()
    response = question_answer_relevant.invoke(
        {"question": state["question"][-1].content, "context": state["context"]}
    )
    return State({"relevant":response.score})
    

def llm_answer_node(state: State):
    prompt = hub.pull("teddynote/rag-prompt-chat-history")
    chain =(
        {
            "question": itemgetter("question"),
            "context": itemgetter("context"),
            "chat_history": itemgetter("chat_history"),
        } | 
        prompt | get_gemini() | StrOutputParser()
        )
    answer = chain.invoke(
       {
            "question": state['question'][-1].content,
            "context": state['context'],
            # "chat_history": ""
            "chat_history": messages_to_history(state['chat_history']),  # 문자열 리스트로 변환
        }
    )
    return {
            "answer": answer,
            "chat_history":[("user" ,state['question'][-1].content),("assistant" , answer)]
            }

def web_node(state:State):
    web_search = TavilySearch()
    search_context = web_search.search(query=state['question'][-1].content)
    return State({"context":search_context})

def search_relevant(state : State):
    question_answer_relevant = GroundednessChecker(
        llm=get_gemini(), target="question-retrieval"
    ).create()
    response = question_answer_relevant.invoke(
        {"question": state["question"][-1].content, "context": state["context"]}
    )
    return State({"relevant":response.score})
   
def is_retriever(state : State)-> Literal["llm_answer_node","web_node"]:
    is_relevant = state.get("relevant","no")
    if is_relevant == "yes":
        return "llm_answer_node"
    return "web_node"

def is_search(state : State)-> Literal["llm_answer_node","re_write_node"]:
    is_relevant = state.get("relevant","no")
    if is_relevant == "yes":
        return "llm_answer_node"
    return "re_write_node"


In [5]:
state_graph = StateGraph(State)
state_graph.add_node('re_write_node',re_write_node)
state_graph.add_node('retriever_node',retriever_node)
state_graph.add_node('retriever_relevant',retriever_relevant)
state_graph.add_node('llm_answer_node',llm_answer_node)
state_graph.add_node('web_node',web_node)
state_graph.add_node('search_relevant',search_relevant)


state_graph.add_edge(START,'re_write_node')
state_graph.add_edge('re_write_node','retriever_node')
state_graph.add_edge('retriever_node','retriever_relevant')
state_graph.add_conditional_edges(
    source='retriever_relevant',
    path=is_retriever,
    path_map={
        "llm_answer_node": "llm_answer_node",  # 관련성이 있으면 답변을 생성합니다.
        "web_node": "web_node",  # 관련성이 없으면 다시 검색합니다.
    },
)
state_graph.add_edge('web_node','search_relevant')
state_graph.add_conditional_edges(
    source='search_relevant',
    path=is_search,
    path_map={
        "llm_answer_node": "llm_answer_node",  # 관련성이 있으면 답변을 생성합니다.
        "re_write_node": "re_write_node",  # 관련성이 없으면 다시 검색합니다.
    },
)
state_graph.add_edge('llm_answer_node',END)



In [6]:
# 메모리 저장소 생성
memory = MemorySaver()
graph = state_graph.compile(checkpointer=memory)
mermaid_code = graph.get_graph().draw_mermaid()
print(mermaid_code)

---
config:
  flowchart:
    curve: linear
---
graph TD;
	__start__([<p>__start__</p>]):::first
	re_write_node(re_write_node)
	retriever_node(retriever_node)
	retriever_relevant(retriever_relevant)
	llm_answer_node(llm_answer_node)
	web_node(web_node)
	search_relevant(search_relevant)
	__end__([<p>__end__</p>]):::last
	__start__ --> re_write_node;
	re_write_node --> retriever_node;
	retriever_node --> retriever_relevant;
	retriever_relevant -.-> llm_answer_node;
	retriever_relevant -.-> web_node;
	search_relevant -.-> llm_answer_node;
	search_relevant -.-> re_write_node;
	web_node --> search_relevant;
	llm_answer_node --> __end__;
	classDef default fill:#f2f0ff,line-height:1.2
	classDef first fill-opacity:0
	classDef last fill:#bfb6fc



In [ ]:
from langchain_core.runnables import RunnableConfig
from langchain_teddynote.messages import stream_graph, invoke_graph, random_uuid


question = '오늘의 날씨'
# 질문 입력
inputs = State({'question':question})

# for event in graph.stream({'question':'엔트로피 투자금액'},config=config,stream_mode="values"):
#     print(event)
# 그래프 실행
stream_graph(graph, inputs, config)


🔄 Node: re_write_node 🔄
- - - - - - - - - - - - - - - - - - - - - - - - - 
오늘의 날씨 정보
🔄 Node: retriever_relevant 🔄
- - - - - - - - - - - - - - - - - - - - - - - - - 

🔄 Node: search_relevant 🔄
- - - - - - - - - - - - - - - - - - - - - - - - - 

🔄 Node: llm_answer_node 🔄
- - - - - - - - - - - - - - - - - - - - - - - - - 
오늘(16일)은 전국 대체로 흐리고 늦은 오후까지 소나기가 내리겠으며, 밤부터 중부지방(강원동해안 제외)과 전북북부에는 비가 오겠습니다. 내일(17일)은 전국이 대체로 흐리고 새벽부터 중부지방(강원동해안 제외)과 전라권에 비가 시작되어 오전부터 강원동해안과 경상권, 제주도에도 비가 오겠으며 밤에 차차 그치겠습니다. 모레(18일)는 전국이 가끔 구름 많겠으나 강원영동은 대체로 흐리고 오전까지 강원영동, 경상권, 제주도에 비가 오겠습니다. 글피(19일)는 전국이 대체로 흐리고 새벽부터 제주도, 오후부터 중부지방(강원영동 제외)과 전라서해안, 경북북부, 밤부터 그 밖의 전라권과 경상서부에 비가 오겠습니다.

- https://www.weather.go.kr/weather/forecast/timeseries.jsp

In [9]:
# 최종 출력

outputs = graph.get_state(config).values
# print(outputs)
print(f'Original Question: {outputs["question"][0].content}')
print(f'Rewritten Question: {outputs["question"][-1].content}')
print("===" * 20)
print(f'Answer:\n{outputs["answer"]}')

Original Question: 오늘의 날씨
Rewritten Question: 오늘의 날씨 정보
Answer:
오늘(16일)은 전국 대체로 흐리고 늦은 오후까지 소나기가 내리겠으며, 밤부터 중부지방(강원동해안 제외)과 전북북부에는 비가 오겠습니다. 내일(17일)은 전국이 대체로 흐리고 새벽부터 중부지방(강원동해안 제외)과 전라권에 비가 시작되어 오전부터 강원동해안과 경상권, 제주도에도 비가 오겠으며 밤에 차차 그치겠습니다. 모레(18일)는 전국이 가끔 구름 많겠으나 강원영동은 대체로 흐리고 오전까지 강원영동, 경상권, 제주도에 비가 오겠습니다. 글피(19일)는 전국이 대체로 흐리고 새벽부터 제주도, 오후부터 중부지방(강원영동 제외)과 전라서해안, 경북북부, 밤부터 그 밖의 전라권과 경상서부에 비가 오겠습니다.

- https://www.weather.go.kr/weather/forecast/timeseries.jsp
